In [ ]:
import time
import toml
import numpy as np
import matplotlib.pyplot as plt

from typing import Dict, Optional
from binance.client import Client
from datetime import datetime, timezone

plt.rcParams["figure.figsize"] = (20,10)
plt.rcParams.update({'font.size': 18})

In [ ]:
cfg = toml.load("configuration.toml")
pkey = cfg["auth"]["pkey"]
skey = cfg["auth"]["skey"]

client = Client(pkey, skey)

## Order Validation

All orders must obey certain rules called Filters (see https://github.com/binance/binance-spot-api-docs/blob/master/rest-api.md#filters). The function `client.get_exchange_info` returns a dictionary that has a list of dictionaries containing information each symbol in the exchange. After searching through the list for the desired dict of information (say for DOGEUSDT), you can retrieve the list of filters which must be obeyed.

Quick recap of the filters. This stuff can be skimmed, then reviewed as needed.

### Price Filter - min / max price
```
{
  "filterType": "PRICE_FILTER",
  "minPrice": "0.00000100",
  "maxPrice": "100000.00000000",
  "tickSize": "0.00000100"
}
```

- `minPrice <= price <= maxPrice`
- `(price - minPrice) % tickSize == 0`


### Percent Price - valid range for price based on prev. trades
```
{
  "filterType": "PERCENT_PRICE",
  "multiplierUp": "1.3000",
  "multiplierDown": "0.7000",
  "avgPriceMins": 5
}
```
- `multiplierDown * weightedAveragePrice <= price <= multiplierUp * weightedAveragePrice`
`weightedAveragePrice` from exchange info for that symbol

### Lot Size - min/max quantity
```
{
  "filterType": "LOT_SIZE",
  "minQty": "0.00100000",
  "maxQty": "100000.00000000",
  "stepSize": "0.00100000"
}
```
- `minQty <= quantity <= maxQty`
- `(quantity-minQty) % stepSize == 0`
If it is an Iceberg order, the quantity for each Iceberg order must satisfy this.

### Min Notional - min/max value
```
{
  "filterType": "MIN_NOTIONAL",
  "minNotional": "0.00100000",
  "applyToMarket": true,
  "avgPriceMins": 5
}
```
- `price * quantity > minNotional`

if `applyToMarket`, Market Orders also must obey this filter. Since market orders have no price, the average price is used over the last `avgPriceMinutes` mins, w/ `avgPriceMinutes == 0` implying that the last price will be used.

### Iceberg Parts - rule for breaking up big orders
```
{
  "filterType": "ICEBERG_PARTS",
  "limit": 10
}
```
Defines the maximum number of parts that an Iceberg Order can have. Number of `ICEBERG_PARTS` is `CEIL(qty / icebergQty)`.

### Market Lot Size - same as Lot size, but for Market Orders

### Max Num Orders - max orders that can be open on a symbol
```
{
  "filterType": "MAX_NUM_ORDERS",
  "maxNumOrders": 25
}
```
There is also `Max num algo orders`, `Max num iceberg orders`, `Exchange max num orders`, `Exchange max num algo orders`, all of which follow the same format.

### Max Position Filter - Defines the allowed max position an account can have on the base asset of a symbol
```
{
  "filterType":"MAX_POSITION",
  "maxPosition":"10.00000000"
}
```
Defined as sum of the account's
1. Free balance of base asset
2. Locked balance of base asset
3. Sum of the quantity of all open BUY orders


# Steps to execute buy/sell order

1. `get_exchange_info` and filter for the desired symbols to find the "Filters" for each symbol
2. Build order conforming to "Filters" for the desired symbol (depending on type of order? Since Market Orders execute instantly on the best possible open bid, we would probably want to go with those)
3. Execute order
4. Profit????

To get the stream of market info, see websocket stuff near bottom of notebook

In [ ]:
exch_info = client.get_exchange_info()

def get_exchange_info_for_symbol(client: Client, symbol: str) -> Optional[Dict]:
    exchange_info = client.get_exchange_info()
    for symbol_data in exchange_info["symbols"]:
        if symbol_data["symbol"] == symbol:
            return symbol_data
    return None

get_exchange_info_for_symbol(client, "DOGEUSDT")

In [ ]:
doge_usd_tick_data = client.get_ticker(symbol="DOGEUSDT")
bid_price = doge_usd_tick_data["lastPrice"]
quantity = 100

print(float(bid_price) * quantity)

min_quantity = f"{round(10.00000000 / float(bid_price) + 1):.8f}"
print(round(1 + 10.00000000 / float(bid_price)), min_quantity)

client.create_test_order(
    symbol="DOGEUSDT",
    side=client.SIDE_BUY,
    type=client.ORDER_TYPE_LIMIT,
    timeInForce=client.TIME_IN_FORCE_GTC,
    quantity=min_quantity,
    price=bid_price
)

# Data Mining

We want that sweet-sweet data. Best endpoint for that is `client.get_historical_trades(symbol="SYMBOL")`, which gives trades in chunks of 500 from some ID (default is most recent).

To get a big ol dataset of historical data,
1. run `historical_trades = client.get_historical_trades(symbol="SYMBOL")` and save the data
2. get minimum id by `min_id = historical_trades[0]["id"]`
3. `goto 1`, but this time include `fromId=str(min_id - 500)`

Since the trade IDs are sequential, you can also totally start from id 0 and work your way up to the most current trades.

In [ ]:
historical_trades = client.get_historical_trades(symbol="DOGEUSDT")

trade_ids = [trd["id"] for trd in historical_trades]

trade_times = [trd["time"] / 100 for trd in historical_trades]
trade_dts = [datetime.fromtimestamp(timestamp, timezone.utc) for timestamp in trade_times]

In [ ]:
plt.plot(trade_dts, [i for i in range(len(trade_dts))])
plt.title("Number of trades over time")
plt.xlabel("trade time")
plt.ylabel("trade number (of the previous 500 trades)")

In [ ]:
print(f"Max dt: {max(np.diff(trade_dts))}\nMin dt: {min(np.diff(trade_dts))}\nMean dt: {(max(trade_dts) - min(trade_dts)) / len(np.diff(trade_dts))}")  # say yes to calcualting the same thing 4 times pointlessly

In [ ]:
fs = [f.total_seconds() for f in np.diff(trade_dts)]
plt.hist(fs, bins=100)  # suppress unecessary output w/ the _ = ...
plt.xlabel(f"time between trades [s]")
plt.ylabel(f"number of trades")

In [ ]:
historical_trades

# Websocket trading

Seems like the way to go - you hand a callback to a stream (1 Hz) of data. It is a rolling window over the previous 24 hrs (e.g. 1 PM yesterday to 1 PM today). Depending on the last price, best bid price, or best ask price, we can decide to make a trade or not. If speed is the primary benefit, Market Orders will take the first possible open bid/ask. It may not give the best price, but will execute instantly, and if we are quick, we will get the best bid/ask price which are listed in the socket stream.

### Ticker-symbol websocket stream format

```json
{
  "e": "24hrTicker",  // Event type
  "E": 123456789,     // Event time
  "s": "BNBBTC",      // Symbol
  "p": "0.0015",      // Price change
  "P": "250.00",      // Price change percent
  "w": "0.0018",      // Weighted average price
  "x": "0.0009",      // First trade(F)-1 price (first trade before the 24hr rolling window)
  "c": "0.0025",      // Last price
  "Q": "10",          // Last quantity
  "b": "0.0024",      // Best bid price
  "B": "10",          // Best bid quantity
  "a": "0.0026",      // Best ask price
  "A": "100",         // Best ask quantity
  "o": "0.0010",      // Open price
  "h": "0.0025",      // High price
  "l": "0.0010",      // Low price
  "v": "10000",       // Total traded base asset volume
  "q": "18",          // Total traded quote asset volume
  "O": 0,             // Statistics open time
  "C": 86400000,      // Statistics close time
  "F": 0,             // First trade ID
  "L": 18150,         // Last trade Id
  "n": 18151          // Total number of trades
}
```

In [ ]:
from binance.websockets import BinanceSocketManager

bm = BinanceSocketManager(client)

class unique_time_dict:
    # Got duplicates from the stream, so create a wrapper
    # to dicts which implement hash/eq so can be added to sets
    def __init__(self, d):
        self._d = d

    def __repr__(self):
        return dict.__repr__(self._d)

    def __hash__(self):
        return self._d["E"]
    
    def __eq__(self, other):
        return self._d["E"] == other["E"]
    
    def __ne__(self, other):
        return not self == other

    def __getitem__(self, k):
        return self._d[k]
    
    def __setitem__(self, k, v):
        self._d[k] = v

s = set()
def cb(msg):
    global s
    s.add(unique_time_dict(msg))

bm.start_symbol_ticker_socket('DOGEUSDT', cb)

In [ ]:
bm.start()
time.sleep(2)
bm.stop_socket("DOGEUSDT")  # somehow this doesn't work

In [ ]:
while True:
    time.sleep(1)
    print(len(s))

from https://python-binance.readthedocs.io/en/latest/websockets.html

Close and exit program

Websockets utilise a reactor loop from the Twisted library. Using the close method above will close the websocket connections but it won’t stop the reactor loop so your code may not exit when you expect.

If you do want to exit then use the stop method from reactor like below.

In [ ]:
# from twisted.internet import reactor
# reactor.stop()